In [1]:
from ipywidgets import interact
from matplotlib import pyplot as plt
import time

In [2]:
# Import tools from the sequence library
#   Timeline is the main element of the simulation kernel
#   QKDNode is a type of quantum node specifically for quantum key distribution
#   Quantum and Classical channels are communication links simulating optical fiber
#   the pair_bb84_protocols is a required function to setup proper key distribution between nodes
from sequence.kernel.timeline import Timeline
from sequence.topology.node import QKDNode
from sequence.components.optical_channel import QuantumChannel, ClassicalChannel
from sequence.protocols.qkd.BB84 import pair_bb84_protocols

In [3]:
# Here is a generic protocol to manage and monitor the BB84 protocol
class KeyManager():
    def __init__(self, timeline, keysize, num_keys):
        self.timeline = timeline
        self.lower_protocols = []
        self.keysize = keysize
        self.num_keys = num_keys
        self.keys = []
        self.times = []
        
    def send_request(self):
        for p in self.lower_protocols:
            p.push(self.keysize, self.num_keys) # interface for BB84 to generate key
            
    def pop(self, msg): # interface for BB84 to return generated keys
        self.keys.append(msg)
        self.times.append(self.timeline.now())

In [6]:
def test(sim_time, keysize):
    """
    sim_time: duration of simulation time (ms)
    keysize: size of generated secure key (bits)
    """
    tl = Timeline(sim_time * 1e9)
    
    # Here, we create nodes for the network (QKD nodes for key distribution)
    # stack_size=1 indicates that only the BB84 protocol should be included
    n1 = QKDNode("n1", tl, stack_size=1)
    n2 = QKDNode("n2", tl, stack_size=1)
    pair_bb84_protocols(n1.protocol_stack[0], n2.protocol_stack[0])
    
    # connect the nodes and set parameters for the fibers
    cc = ClassicalChannel("cc_n1_n2", tl, distance=1e3)
    cc.set_ends(n1, n2)
    qc = QuantumChannel("qc_n1_n2", tl, attenuation=1e-5, distance=1e3,
                       polarization_fidelity=0.97)
    qc.set_ends(n1, n2)
    
    # instantiate our written keysize protocol
    km1 = KeyManager(tl, keysize, 10)
    km1.lower_protocols.append(n1.protocol_stack[0])
    n1.protocol_stack[0].upper_protocols.append(km1)
    km2 = KeyManager(tl, keysize, 10)
    km2.lower_protocols.append(n2.protocol_stack[0])
    n2.protocol_stack[0].upper_protocols.append(km2)
    
    # start simulation and record timing
    tl.init()
    km1.send_request()
    tick = time.time()
    tl.run()
    print("execution time %.2f sec" % (time.time() - tick))
    
    # display our collected metrics
    plt.plot(km1.times, range(1, len(km1.keys) + 1), marker="o")
    plt.xlabel("Simulation time (ps)")
    plt.ylabel("Number of Completed Keys")
    plt.show()
    print("key error rates:")
    for i, e in enumerate(n1.protocol_stack[0].error_rates):
        print("\tkey {}:\t{}%".format(i + 1, e * 100))

In [7]:
# Create and run the simulation
interactive_plot = interact(test, sim_time=(100, 1000, 100), keysize=[128, 256, 512])
interactive_plot

interactive(children=(IntSlider(value=500, description='sim_time', max=1000, min=100, step=100), Dropdown(desc…

<function __main__.test(sim_time, keysize)>